There are many deep-learning libraries in Julia. One of the most used one is Flux.jl  
*"Relax! Flux is the ML library that doesn't make you tensor https://fluxml.ai/ "*

In [ ]:
using Flux
model = Chain(
  Dense(10, 5, σ),
  Dense(5, 2),
  softmax)

model(rand(10))

In [ ]:
using Flux: Params

W = rand(2, 5)
b = rand(2)

predict(x) = W*x .+ b
loss(x, y) = sum((predict(x) .- y).^2)

x, y = rand(5), rand(2) # Dummy data
l = loss(x, y) # ~ 3

θ = Params([W, b])
grads = Flux.gradient(() -> loss(x, y), θ)

In [ ]:
@show loss(x, y)
η = 0.01 # Learning Rate
for p in (W, b)
  p .-= η * grads[p]
end
@show loss(x, y)

Running this will alter the parameters W and b and our loss should go down. Flux provides a more general way to do optimiser updates like this.

In [ ]:
using Flux.Optimise: update!
@show loss(x, y)
opt = Descent(0.01) # Gradient descent with learning rate 0.1

for p in (W, b)
  update!(opt, p, grads[p])
end
@show loss(x, y)

# Training
To actually train a model we need three things:

- A objective function, that evaluates how well a model is doing given some input data.
- A collection of data points that will be provided to the objective function.
- An optimiser that will update the model parameters appropriately.
With these we can call Flux.train!:  
`Flux.train!(objective, params, data, opt)`

In [ ]:
m = Chain(
  Dense(784, 32, σ),
  Dense(32, 10), softmax)

loss(x, y) = Flux.mse(m(x), y)
ps = Flux.params(m)

# later
Flux.train!(loss, ps, data, opt)

# Datasets
The data argument provides a collection of data to train with (usually a set of inputs `x` and target outputs `y`). For example, here's a dummy data set with only one data point:

In [ ]:
x = rand(784)
y = rand(10)
dataset = [(x, y)]

Flux.train! will call `loss(x, y)`, calculate gradients, update the weights and then move on to the next data point if there is one. We can train the model on the same data three times:

In [ ]:
dataset = [(x, y), (x, y), (x, y)]
# Or equivalently
dataset = Iterators.repeated((x, y), 3)

It's common to load the xs and ys separately. In this case you can use zip:

In [ ]:
xs = [rand(784), rand(784), rand(784)]
ys = [rand( 10), rand( 10), rand( 10)]
dataset = zip(xs, ys)

Note that, by default, train! only loops over the data once (a single "epoch"). A convenient way to run multiple epochs from the REPL is provided by `@epochs`.

# Example -- Time series clustering 
Say we have a bunch of $n$ time series of length $T$ collected in matrix $A \in \mathbb{R^{T\times m}}$

Can we decompose $A$ as a combination of "basis time series"?

This is exactly what SVD does $A = U(SV^T)$. If we keep only the $k$ largest singular value/vector pairs, we have a low-rank approximation (the number of basis time series is lower than the number of measured time series.

What if we want a sparse decomposition?
$$A = WH$$ where $h$ is sparse.

What if we want each time series $a_i = W h_i$ to be not only sparse in $h_i$, but that $h_i$ lives on the probability simplex
$$h_i \geq 0, \quad \sum h_i = 1$$
This would allow us to say that $a_i$ is 10% of one base vector and 90% of another one, etc.

Dirichlet process prior on $h_i \sim Dirichlet(\alpha)$

One can solve this problem using Bayesian non-parametrics, but it takes forever (I have tried)

One can also just maximize
$$|| A - WH|| - (\alpha -1)\sum H$$

To make sure $h_i$ stays on the probability simplex, we optimize over $\bar h_i$ and calculate $h_i = softmax(\bar h_i)$

We start by creating some data

In [ ]:
m,n,k = 20,10,3

W = randn(m,k)
H = softmax(5randn(k,n))
A = W*H
H

We can try the SVD method first

In [ ]:
function rank_k_svd(A,k)
    s = svd(A)
    Wsvd, Hsvd = s.U[:,1:k], s.Vt[1:k,:]
    Asvd = Wsvd*Diagonal(s.S[1:k])*Hsvd
    Wsvd,Hsvd,Asvd
end

Wsvd,Hsvd,Asvd = rank_k_svd(A,k)
@assert norm(A-Asvd)/norm(A) < 1e-10
Hsvd

The resulting $H$ is dense and hard to interpret

We now set up and solve the stated optimization problem using Flux 

In [ ]:
α = 0.2
Wh,Hh = randn(m,k+1), randn(k+1,n)
p = Params((Wh,Hh))
dir(H,α) = (α-1)*sum(log, H)
function cost()
    H = softmax(Hh)
    norm(A-Wh*H) - dir(H,α) # Negative likelihood
end

We then create an optimizer and run gradient-based training for a number of iterations

In [ ]:
opt = ADAM()

In [ ]:
@time for i = 1:2000
    gs = Flux.gradient(cost, p)
    Flux.Optimise.update!(opt, p, gs)
end
Ah = (Wh*softmax(Hh))
@show norm(A-Asvd)/norm(A)
@show norm(A-Ah)/norm(A)

In [ ]:
heatmap(A, layout=2, ylabel="Time", xlabel="Time-series index"); heatmap!(Ah, subplot=2)

How well does our decomposition approximate the original time series?

In [ ]:
plot(A, c=:red, layout=10); plot!(Ah, c=:blue, legend=false)

Just for fun, let's look at the basis functions

In [ ]:
plot(Wh, layout=size(Wh,2))

Now, let's see if the resulting $H$ is sparse

In [ ]:
plot(heatmap(softmax(Hh)), heatmap(Hsvd))

To see if we have used too many basis functions, we see if there are strong correlations between them.

In [ ]:
heatmap(abs.(cor(Hh')))